In [1]:
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim

from torch.utils import data
from tqdm import tqdm_notebook as tqdm

In [2]:
#create network
class block(nn.Module):
    def __init__(self,planes):
        super(block, self).__init__()
        self.fc1 = nn.Linear(planes,planes)
        self.fc2 = nn.Linear(planes,planes)
        self.fc3 = nn.Linear(planes,planes)
        self.fc4 = nn.Linear(planes,planes)
        self.relu = nn.ReLU()
    def forward(self,a,b,c,d,x0,x1=None,x2=None,x3=None):

        x_1 = self.fc1(x0)
        x_2 = self.fc2(x1)
        x_3 = self.fc3(x2)
        x_4 = self.fc4(x3)
        x5= a*x_1 + b*x_2 + c*x_3+ d*x_4
        x5 = self.relu(x5)
        return x_2,x_3,x_4,x5

class Net(nn.Module):
    def __init__(self,block,planes):
        super(Net,self).__init__()
        self.a = nn.Parameter(torch.Tensor(1).uniform_(0.5, 1),requires_grad=True)
        self.b = nn.Parameter(torch.Tensor(1).uniform_(-1, 1),requires_grad=True)
        self.c = nn.Parameter(torch.Tensor(1).uniform_(0,0.5),requires_grad=True)
        self.block1 = block(planes)
        self.block2 = block(planes)
        self.block3 = block(planes)
        self.block4 = block(planes)
        self.block5 = block(planes)
        self.block6 = block(planes)
 
        self.fc = nn.Linear(planes,4)
    def forward(self,x):
        x_1,x_2,x_3,out = self.block1(1+self.c,-2*self.c,self.c,self.b,x)
        x_1,x_2,x_3,out = self.block2(1+self.c,-2*self.c,self.c,self.b,x_1,x_2,x_3,out)
        x_1,x_2,x_3,out = self.block3(1+self.c,-2*self.c,self.c,self.b,x_1,x_2,x_3,out)
        x_1,x_2,x_3,out = self.block4(1+self.c,-2*self.c,self.c,self.b,x_1,x_2,x_3,out)
        x_1,x_2,x_3,out = self.block5(1+self.c,-2*self.c,self.c,self.b,x_1,x_2,x_3,out)
        _,_,_,out = self.block6(1+self.c,-2*self.c,self.c,self.b,x_1,x_2,x_3,out)
        out = self.fc(out)
        return out

In [3]:
#set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
#hyperparameters
planes = 16
num_epochs = 10
learning_rate = 0.001
batch_size = 1
time_step= 4
train_num = 20000

In [5]:

#load data
def sample(time_step,train_num): 
    sample_num = 20000
    t=np.linspace(0,2*np.pi, sample_num).reshape(sample_num,1)
    Sint=np.sin(t)
    Cost=np.cos(t)
    X = np.hstack((Sint,Cost))
    X = np.hstack((X,-Cost))
    X = np.hstack((X,Sint))
    Y_t = np.hstack((Sint,-Cost))
    Y_t = np.hstack((Y_t,Cost))
    Y_t = np.hstack((Y_t,Sint))
    xTrainDataset = X[0:train_num]
    yTrainDataset = Y_t[0:train_num]
    xTrain = []
    yTrain = []
    for i in range(time_step,  train_num):
        data = xTrainDataset[i-time_step]
        for j in range(1,time_step):
            data = np.hstack((data,xTrainDataset[i-time_step+j]))
        xTrain.append(data)
    for i in range(time_step, train_num):
        yTrain.append(yTrainDataset[i])
    return xTrain,yTrain

x,y=sample(time_step,train_num)

tensor_x = torch.Tensor(x) # transform to torch tensor
tensor_y = torch.Tensor(y)
train_dataset = data.TensorDataset(tensor_x,tensor_y) # create  datset
train_loader = data.DataLoader(train_dataset) # create  dataloader
x[0][0:4]


array([ 0.,  1., -1.,  0.])

In [6]:
#initialize network
model = Net(block,planes).to(device)

In [7]:
#loss and optimizer
critirion = nn.L1Loss()
optimizer = optim.Adam(params=model.parameters(),lr=learning_rate)

In [8]:

for epoch in range(num_epochs):
    print(1+model.c,-2*model.c,model.c)
    for tdata,target in train_loader:
        tdata = tdata.to(device=device)
        target = target.to(device=device)
        
        scores = model(tdata)
        loss = critirion(scores,target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f'epoch: {epoch} loss: {loss.item():10.8f}')


        

tensor([1.1236], device='cuda:0', grad_fn=<AddBackward0>) tensor([-0.2472], device='cuda:0', grad_fn=<MulBackward0>) Parameter containing:
tensor([0.1236], device='cuda:0', requires_grad=True)
epoch: 0 loss: 0.00176498
tensor([1.1675], device='cuda:0', grad_fn=<AddBackward0>) tensor([-0.3349], device='cuda:0', grad_fn=<MulBackward0>) Parameter containing:
tensor([0.1675], device='cuda:0', requires_grad=True)
epoch: 1 loss: 0.00176379
tensor([1.1962], device='cuda:0', grad_fn=<AddBackward0>) tensor([-0.3924], device='cuda:0', grad_fn=<MulBackward0>) Parameter containing:
tensor([0.1962], device='cuda:0', requires_grad=True)
epoch: 2 loss: 0.00240406
tensor([1.1952], device='cuda:0', grad_fn=<AddBackward0>) tensor([-0.3903], device='cuda:0', grad_fn=<MulBackward0>) Parameter containing:
tensor([0.1952], device='cuda:0', requires_grad=True)
epoch: 3 loss: 0.00167682
tensor([1.1121], device='cuda:0', grad_fn=<AddBackward0>) tensor([-0.2242], device='cuda:0', grad_fn=<MulBackward0>) Paramet